In [1]:
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
import tensorflow as tf
import numpy as np
import random
import json

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'chatbot',
   'patterns': ['Who built this chatbot?',
    'Tell me about Chatbot',
    'What is this chatbot name?'],
   'responses': ['Hi, I am Chatbot designed by Anurag Singh.',
    'Thanks for asking. I am designed by Anurag Singh.',
    'I am a chatbot.']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?'],
   'responses': ["We are fro

In [4]:
words = []
classes = []
documents = []
ignore = ['?']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
# Perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

24 documents
7 classes ['about', 'chatbot', 'connect', 'goodbye', 'greeting', 'location', 'thanks']
46 unique stemmed words ["'s", 'about', 'account', 'address', 'ani', 'anyon', 'are', 'built', 'bye', 'can', 'chatbot', 'connect', 'day', 'give', 'good', 'goodby', 'hello', 'help', 'hi', 'how', 'i', 'is', 'later', 'link', 'locat', 'me', 'media', 'name', 'out', 'reach', 'see', 'social', 'tell', 'thank', 'that', 'there', 'thi', 'to', 'way', 'we', 'what', 'where', 'who', 'you', 'your', 'yourself']


In [6]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-6-0222230fb184>:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [7]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))
model.compile(tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy']) 

In [8]:
model.fit(np.array(train_x), np.array(train_y), epochs=1000, batch_size=8, verbose=1)
model.save("model.pkl")

Epoch 1/1000
3/3 [==============================] - 0s 1ms/step - loss: 2.0611 - accuracy: 0.0833
Epoch 2/1000
3/3 [==============================] - 0s 1ms/step - loss: 2.0399 - accuracy: 0.1250
Epoch 3/1000
3/3 [==============================] - 0s 2ms/step - loss: 2.0214 - accuracy: 0.1250
Epoch 4/1000
3/3 [==============================] - 0s 2ms/step - loss: 2.0045 - accuracy: 0.1667
Epoch 5/1000
3/3 [==============================] - 0s 2ms/step - loss: 1.9858 - accuracy: 0.1667
Epoch 6/1000
3/3 [==============================] - 0s 2ms/step - loss: 1.9694 - accuracy: 0.1667
Epoch 7/1000
3/3 [==============================] - 0s 2ms/step - loss: 1.9518 - accuracy: 0.2083
Epoch 8/1000
3/3 [==============================] - 0s 975us/step - loss: 1.9358 - accuracy: 0.2083
Epoch 9/1000
3/3 [==============================] - 0s 2ms/step - loss: 1.9196 - accuracy: 0.2083
Epoch 10/1000
3/3 [==============================] - 0s 2ms/step - loss: 1.9037 - accuracy: 0.2083
Epoch 11/1000
3/3

3/3 [==============================] - 0s 2ms/step - loss: 0.6898 - accuracy: 1.0000
Epoch 84/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.6773 - accuracy: 1.0000
Epoch 85/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.6628 - accuracy: 1.0000
Epoch 86/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.6508 - accuracy: 1.0000
Epoch 87/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.6371 - accuracy: 1.0000
Epoch 88/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.6246 - accuracy: 1.0000
Epoch 89/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.6124 - accuracy: 1.0000
Epoch 90/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.5990 - accuracy: 1.0000
Epoch 91/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.5870 - accuracy: 1.0000
Epoch 92/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.5744 - accuracy: 1.0000
Epoch 93/1000
3/3 [=====

Epoch 165/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.1023 - accuracy: 1.0000
Epoch 166/1000
3/3 [==============================] - 0s 997us/step - loss: 0.1000 - accuracy: 1.0000
Epoch 167/1000
3/3 [==============================] - 0s 996us/step - loss: 0.0978 - accuracy: 1.0000
Epoch 168/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0958 - accuracy: 1.0000
Epoch 169/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.0938 - accuracy: 1.0000
Epoch 170/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.0919 - accuracy: 1.0000
Epoch 171/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0900 - accuracy: 1.0000
Epoch 172/1000
3/3 [==============================] - 0s 997us/step - loss: 0.0882 - accuracy: 1.0000
Epoch 173/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.0863 - accuracy: 1.0000
Epoch 174/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.0846 - accuracy: 1

3/3 [==============================] - 0s 2ms/step - loss: 0.0268 - accuracy: 1.0000
Epoch 247/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.0265 - accuracy: 1.0000
Epoch 248/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.0261 - accuracy: 1.0000
Epoch 249/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0258 - accuracy: 1.0000
Epoch 250/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0255 - accuracy: 1.0000
Epoch 251/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0252 - accuracy: 1.0000
Epoch 252/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0249 - accuracy: 1.0000
Epoch 253/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.0246 - accuracy: 1.0000
Epoch 254/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.0243 - accuracy: 1.0000
Epoch 255/1000
3/3 [==============================] - 0s 997us/step - loss: 0.0240 - accuracy: 1.0000
Epoch 256/100

3/3 [==============================] - 0s 2ms/step - loss: 0.0117 - accuracy: 1.0000
Epoch 328/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0116 - accuracy: 1.0000
Epoch 329/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.0115 - accuracy: 1.0000
Epoch 330/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.0114 - accuracy: 1.0000
Epoch 331/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0113 - accuracy: 1.0000
Epoch 332/1000
3/3 [==============================] - 0s 993us/step - loss: 0.0112 - accuracy: 1.0000
Epoch 333/1000
3/3 [==============================] - 0s 954us/step - loss: 0.0111 - accuracy: 1.0000
Epoch 334/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.0110 - accuracy: 1.0000
Epoch 335/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.0109 - accuracy: 1.0000
Epoch 336/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.0108 - accuracy: 1.0000
Epoch 337/1

...

3/3 [==============================] - 0s 2ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 655/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 656/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 657/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 658/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 659/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 660/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 661/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 662/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 663/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 664/1000


3/3 [==============================] - 0s 2ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 737/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 738/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 739/1000
3/3 [==============================] - 0s 3ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 740/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 741/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 742/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 743/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 744/1000
3/3 [==============================] - 0s 3ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 745/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 746/1000


3/3 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 819/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 820/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 821/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 822/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 823/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 824/1000
3/3 [==============================] - 0s 3ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 825/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 826/1000
3/3 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 827/1000
3/3 [==============================] - 0s 1ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 828/1000


3/3 [==============================] - 0s 2ms/step - loss: 8.4547e-04 - accuracy: 1.0000
Epoch 898/1000
3/3 [==============================] - 0s 2ms/step - loss: 8.4300e-04 - accuracy: 1.0000
Epoch 899/1000
3/3 [==============================] - 0s 2ms/step - loss: 8.4048e-04 - accuracy: 1.0000
Epoch 900/1000
3/3 [==============================] - 0s 2ms/step - loss: 8.3797e-04 - accuracy: 1.0000
Epoch 901/1000
3/3 [==============================] - 0s 2ms/step - loss: 8.3545e-04 - accuracy: 1.0000
Epoch 902/1000
3/3 [==============================] - 0s 2ms/step - loss: 8.3299e-04 - accuracy: 1.0000
Epoch 903/1000
3/3 [==============================] - 0s 2ms/step - loss: 8.3049e-04 - accuracy: 1.0000
Epoch 904/1000
3/3 [==============================] - 0s 2ms/step - loss: 8.2828e-04 - accuracy: 1.0000
Epoch 905/1000
3/3 [==============================] - 0s 1ms/step - loss: 8.2553e-04 - accuracy: 1.0000
Epoch 906/1000
3/3 [==============================] - 0s 2ms/step - loss: 8.232

3/3 [==============================] - 0s 2ms/step - loss: 6.7547e-04 - accuracy: 1.0000
Epoch 976/1000
3/3 [==============================] - 0s 2ms/step - loss: 6.7358e-04 - accuracy: 1.0000
Epoch 977/1000
3/3 [==============================] - 0s 2ms/step - loss: 6.7179e-04 - accuracy: 1.0000
Epoch 978/1000
3/3 [==============================] - 0s 2ms/step - loss: 6.6989e-04 - accuracy: 1.0000
Epoch 979/1000
3/3 [==============================] - 0s 2ms/step - loss: 6.6804e-04 - accuracy: 1.0000
Epoch 980/1000
3/3 [==============================] - 0s 2ms/step - loss: 6.6613e-04 - accuracy: 1.0000
Epoch 981/1000
3/3 [==============================] - 0s 2ms/step - loss: 6.6439e-04 - accuracy: 1.0000
Epoch 982/1000
3/3 [==============================] - 0s 2ms/step - loss: 6.6241e-04 - accuracy: 1.0000
Epoch 983/1000
3/3 [==============================] - 0s 2ms/step - loss: 6.6067e-04 - accuracy: 1.0000
Epoch 984/1000
3/3 [==============================] - 0s 2ms/step - loss: 6.587

In [9]:
import pickle
pickle.dump( {'words':words, 'classes':classes}, open( "training_data", "wb" ) )

In [10]:
from keras.models import load_model
model = load_model("model.pkl")

In [11]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']

In [12]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [13]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
    bag=np.array(bag)
    return(bag)

In [14]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    bag = bow(sentence, words)
    results = model.predict(np.array([bag]))
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results[0]) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [15]:
response('Where are you located?')

You can visit India to meet us


In [16]:
response('tell me something about yourself')

I am a engineering student from the University of Petroleum and Energy Studies, Dehradun, pursuing a degree in B.Tech computer science specialization in Artificial Intelligence and Machine Learning


In [17]:
response('have a good day')

Good to see you again


In [18]:
response('good night')

Good to see you again


In [19]:
response('who bulid this chatbot')

Hi, I am Chatbot designed by Anurag Singh.
